In [1]:
import os
import warnings
from dotenv import load_dotenv

import faiss
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_core.documents import Document
from langchain_community.document_loaders import TextLoader
from langchain_community.docstore.in_memory import InMemoryDocstore

In [2]:
# .env 로드
load_dotenv()
# os.getenv("OPENAI_API_KEY")

# 경고 무시
warnings.filterwarnings("ignore")

In [5]:
# 임베딩 모델 크기 조회
# pip install langchain-huggingface sentence-transformers
from langchain_huggingface.embeddings import HuggingFaceEmbeddings

# model_name = "intfloat/multilingual-e5-large-instruct"
# model_name = "intfloat/multilingual-e5-large"
model_name = "BAAI/bge-m3"

hf_embeddings = HuggingFaceEmbeddings(
    model_name = model_name,
    cache_folder="C:/cache",
    model_kwargs={"device": "cuda"},             # cuda, cpu, mps
    encode_kwargs={"normalize_embeddings": True},
)

# 임베딩 차원 크기 조회 (FAISS 필요)
dimension_size = len(hf_embeddings.embed_query("hello world"))
print(dimension_size)       # 3072

No sentence-transformers model found with name BAAI/bge-m3. Creating a new one with mean pooling.


OSError: We couldn't connect to 'https://huggingface.co' to load the files, and couldn't find them in the cached files.
Check your internet connection or see how to run the library in offline mode at 'https://huggingface.co/docs/transformers/installation#offline-mode'.

In [12]:
# 1단계 : 문서 로드
loader = PyMuPDFLoader("SPRI_AI_Brief_2023년12월호_F.pdf")
docs = loader.load()

# 2단계 : 문서 분할
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=50)
split_documents = text_splitter.split_documents(docs)

# 3단계 : 임베딩
embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

# 4단계 : 벡터스토어 생성 및 로컬저장 후 로드
db = FAISS(
    embedding_function=embeddings,
    index=faiss.IndexFlatL2(dimension_size),
    docstore=InMemoryDocstore(),
    index_to_docstore_id={}
)
db = FAISS.from_documents(documents=split_documents, embedding=embeddings)
db.save_local(folder_path="faiss_db", index_name="faiss_index")

loaded_db = FAISS.load_local(
    folder_path='faiss_db',
    index_name='faiss_index',
    embeddings=embeddings,
    allow_dangerous_deserialization=True,
)

In [13]:
loaded_db.index_to_docstore_id

{0: '60689d91-c626-4258-ab74-06d05dc32b25',
 1: 'f4f74467-274f-4e6e-accc-36d6be5a8880',
 2: '52c89384-b52c-49c6-b13e-d57f0f14f236',
 3: '899569fc-0635-426b-8c5d-0e7b5c311dfc',
 4: '1e22100e-9ccc-44ce-a25f-2966a24778bc',
 5: '2fbb27ef-848d-4f92-99b6-e3a50f98f850',
 6: 'def5f254-5494-4cfa-a44f-93e2834683a2',
 7: 'e2bd3a7d-2a82-44bf-a22a-72be919a2acd',
 8: '5c3023de-355c-4bfe-aaba-ade236eb9587',
 9: '55fa480b-ee77-44a2-9694-e8ecc48a1977',
 10: '939cdbe4-70fa-44a1-a874-f1bfa0cd7c42',
 11: '0674cadb-1e68-4471-8aa4-d7211c60c1c7',
 12: '4f471dc8-785f-46b5-8a94-73e759e0b782',
 13: '2f1ab751-e081-4645-b76d-8b4f3767baf9',
 14: '2955a465-efd2-441e-b0b3-faa14cd72b49',
 15: '0c85ac09-abe8-434f-ae37-c3046195d41e',
 16: '24f5eac6-79a2-453f-8fd9-e0e061e96860',
 17: 'ad513b08-9ab8-47aa-89d1-756052813c50',
 18: '3bd52a7e-b30b-498f-9d0a-f3587f6f9048',
 19: 'e2bb1347-2001-4075-ad07-78b088799bf8',
 20: '9000b99c-de85-4953-9141-2ebaa027a2bc',
 21: 'c0bbe47a-10ad-45b2-9428-38160be81384',
 22: 'b9fe5579-657d-

In [14]:
# 5단계 : 검색기 생성 (Retriever)
retriever = loaded_db.as_retriever()

# 6단계 : 프롬프트
prompt = PromptTemplate.from_template(
    """You are an assistant for question-answering tasks.
Use the following pieces of retrieved context to answer the question.
If you don't know the answer, just say that you don't know.
Answer in Korean.

#Question:
{question}

#Context:
{context}

#Answer:"""
)

# 7단계 : LLM 생성
llm = ChatOpenAI(model_name="gpt-5-nano", temperature=0, api_key=os.getenv("OPENAI_API_KEY"))

# 8단계 : chain
chain = (
    {"context":retriever, "question":RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [16]:
# pip install langchain_teddynote
from langchain_teddynote.messages import stream_response

question = "삼성전자가 자체 개발한 AI 의 이름은?"
response = chain.stream(question)
stream_response(response)

삼성 가우스